# Exploring and cleaning NYC Airbnb Dataset

## 1. Exploring

In [50]:
import numpy as np
import pandas as pd
import requests
import src.cleaning_utils as cu


In [51]:
Abb_NY = pd.read_csv("data/AB_NYC_2019.csv",encoding = "ISO-8859-1")

In [52]:
Abb_NY.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [53]:
Abb_NY.shape

(48895, 16)

In [54]:
# We observe our columns in order to see which ones could be usefull for our analysis.
Abb_NY.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [55]:
# What type of data do we have in each column?
Abb_NY_dtype=Abb_NY.dtypes
Abb_NY_dtype

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [56]:
# We calculate null ratio for each columns.
perc_null_col = Abb_NY.isnull().sum().apply(lambda x: x/Abb_NY.shape[0]).sort_values(ascending=False)
perc_null_col

reviews_per_month                 0.205583
last_review                       0.205583
host_name                         0.000429
name                              0.000327
availability_365                  0.000000
calculated_host_listings_count    0.000000
number_of_reviews                 0.000000
minimum_nights                    0.000000
price                             0.000000
room_type                         0.000000
longitude                         0.000000
latitude                          0.000000
neighbourhood                     0.000000
neighbourhood_group               0.000000
host_id                           0.000000
id                                0.000000
dtype: float64

In [57]:
# We use the same method of the columns for analyze the rows.
perc_null_row=Abb_NY.isnull().sum(axis=1).apply(lambda x: x/Abb_NY.shape[1]).sort_values(ascending=False)
perc_null_row

16071    0.1875
27777    0.1875
38992    0.1875
6605     0.1875
2854     0.1875
          ...  
29611    0.0000
29610    0.0000
29608    0.0000
29607    0.0000
0        0.0000
Length: 48895, dtype: float64

In [58]:
# And finally we look for duplicate visits.
Abb_NY.duplicated().sum()

0

## 2.Cleaning

In [59]:
drop_cols = ["id","name","host_id","host_name","reviews_per_month","calculated_host_listings_count","availability_365"]
Abb_NY_clean = Abb_NY.drop(drop_cols, axis =1)
Abb_NY_clean

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19
...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN
48892,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN


In [60]:
Abb_NY_clean["last_review"].value_counts(dropna=False)

NaN           10052
2019-06-23     1413
2019-07-01     1359
2019-06-30     1341
2019-06-24      875
              ...  
2013-05-24        1
2014-05-05        1
2013-08-08        1
2014-11-14        1
2016-01-04        1
Name: last_review, Length: 1765, dtype: int64

In [61]:
Abb_NY_clean['year'] = pd.DatetimeIndex(Abb_NY_clean['last_review']).year
Abb_NY_clean

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,year
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,2018.0
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,2019.0
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,2019.0
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,2018.0
...,...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN
48891,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN
48892,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN
48893,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN


In [62]:
drop_col = ["last_review"]
Abb_NY_clean = Abb_NY_clean.drop(drop_col, axis =1)

In [63]:
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
NaN       10052
2018.0     6050
2017.0     3205
2016.0     2707
2015.0     1393
2014.0      199
2013.0       48
2012.0       25
2011.0        7
Name: year, dtype: int64

In [64]:
Abb_NY_clean["year"] = Abb_NY_clean[Abb_NY_clean["year"]>2015]["year"]
Abb_NY_clean

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018.0
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019.0
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018.0
...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN
48892,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN


In [47]:
Abb_NY_clean = Abb_NY_clean[Abb_NY_clean["year"].notnull()]
Abb_NY_clean

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018.0
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019.0
3,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018.0
5,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019.0
...,...,...,...,...,...,...,...,...,...
48782,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,1,2019.0
48790,Queens,Flushing,40.75104,-73.81459,Private room,45,1,1,2019.0
48799,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,1,2019.0
48805,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,2,2019.0


In [48]:
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
2018.0     6050
2017.0     3205
2016.0     2707
Name: year, dtype: int64

In [49]:
Abb_NY_clean["neighbourhood_group"].value_counts(dropna=False)

Manhattan        15834
Brooklyn         15702
Queens            4457
Bronx              869
Staten Island      309
Name: neighbourhood_group, dtype: int64

In [50]:
Abb_NY_clean["price"].value_counts(dropna=False)

150    1502
100    1434
50     1147
60     1107
75     1034
       ... 
442       1
474       1
506       1
332       1
242       1
Name: price, Length: 576, dtype: int64

In [52]:
Abb_NY_clean.to_csv("output/Abb_NY_clean_output.csv", index = False)